# Trabajo integrador - Parte 1
## Python y Numpy

**Nombre**: Juan Cruz Ferreyra

In [1]:
import numpy as np

## Ejercicio 1

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||_{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [31]:
def _validate_p(p):
    is_valid = p in [0, 1, 2, np.inf]
    if not is_valid:
        raise ValueError("p must be either 0, 1, 2, np.inf.")


def compute_norm(x: np.array, p: int = 1):
    _validate_p(p)
    if p == 0:
        return np.count_nonzero(x, axis=1)
    elif p == np.inf:
        return np.max(np.abs(x), axis=1)
    else:
        return np.sum(np.abs(x) ** p, axis=1) ** (1 / p)

In [22]:
x = np.array([[0, 3, 4], [3, 12, 4]])

print(f"l0:   {compute_norm(x, 0)}")
print(f"l1:   {compute_norm(x, 1)}")
print(f"l2:   {compute_norm(x, 2)}")
print(f"linf: {compute_norm(x, np.inf)}")

l0:   [2 3]
l1:   [ 7. 19.]
l2:   [ 5. 13.]
linf: [ 4 12]


## Ejercicio 2

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [39]:
def _validate_arrays(prediction: np.array, truth: np.array):
    is_valid = prediction.shape == truth.shape
    if not is_valid:
        raise ValueError("prediction and truth arrays must have the same shape.")


def _validate_metric(metric: str):
    is_valid = metric in ["precision", "recall", "accuracy"]
    if not is_valid:
        raise ValueError("p must be either 0, 1, 2, np.inf")


def _cm_metric(x1: np.array, x2: np.array, metric: str):
    if metric == "tp":
        return np.sum((x1 == x2) & (x1 == 1))
    elif metric == "fp":
        return np.sum((x1 != x2) & (x1 == 1))
    elif metric == "tn":
        return np.sum((x1 == x2) & (x1 == 0))
    elif metric == "fn":
        return np.sum((x1 != x2) & (x1 == 0))


def calculate_metric(prediction: np.array, truth: np.array, metric: str = "precision"):
    _validate_arrays(prediction, truth)
    _validate_metric(metric)

    if metric == "precision":
        tp = _cm_metric(prediction, truth, "tp")
        fp = _cm_metric(prediction, truth, "fp")
        return tp / (tp + fp)
    elif metric == "recall":
        tp = _cm_metric(prediction, truth, "tp")
        fn = _cm_metric(prediction, truth, "fn")
        return tp / (tp + fn)
    else:
        tp = _cm_metric(prediction, truth, "tp")
        fp = _cm_metric(prediction, truth, "fp")
        tn = _cm_metric(prediction, truth, "tn")
        fn = _cm_metric(prediction, truth, "fn")
        return (tp + tn) / (tp + fp + tn + fn)

In [40]:
prediction = np.array([1,1,1,1,0,0,1,1,0,0])
truth = np.array([1,1,0,1,1,1,0,0,0,1])

In [46]:
print(f"Precision: {calculate_metric(prediction, truth, 'precision')}")
print(f"Recall: {calculate_metric(prediction, truth, 'recall')}")
print(f"Accuracy: {calculate_metric(prediction, truth, 'accuracy')}")

Precision: 0.5
Recall: 0.5
Accuracy: 0.4


## Ejercicio 3

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [86]:
def _validate_input(X: np.array, y: np.array):
    is_valid = X.shape[0] == y.shape[0]
    if not is_valid:
        raise ValueError("Num of observations in 'X' should equal 'y' length.")
    
def split(
    X_input,
    y_input,
    train_size=0.7,

    val_size=0.15,
    test_size=0.15,
    random_state=42,
    shuffle=True,
):
    _validate_input(X_input, y_input)
    
    np.random.seed(random_state)

    if shuffle:
        idx_permutation = np.random.permutation(len(X_input))

        X_input = X_input[idx_permutation]
        y_input = y_input[idx_permutation]

    train_n = int(train_size * len(X_input))
    val_n = int(val_size * len(X_input))

    return (
        (X_input[:train_n], y_input[:train_n]),
        (
            X_input[train_n : (train_n + val_n)],
            y_input[train_n : (train_n + val_n)],
        ),
        (X_input[(train_n + val_n) :], y_input[(train_n + val_n) :]),
    )

In [82]:
# Generate dummy data.
X = np.vstack([np.arange(10) * (i + 1) for i in range(100)])
y = np.mean(X, axis=1)

In [87]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = split(X, y)